# Exercise 02 : Join

## Create a connection to the database using the library sqlite3


In [11]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("../data/checking-logs.sqlite")

## Create a new table datamart in the database by joining the tables pageviews and checker using only one query


In [12]:
create_datamart_sql = """
CREATE TABLE IF NOT EXISTS datamart AS
SELECT
  c.uid,
  c.labname,
  c.timestamp      AS first_commit_ts,
  pv.datetime      AS first_view_ts
FROM
  checker AS c
LEFT JOIN
  (
    SELECT
      uid,
      MIN(datetime) AS datetime
    FROM pageviews
    WHERE uid LIKE 'user_%'
    GROUP BY uid
  ) AS pv
ON c.uid = pv.uid
WHERE
  c.status   = 'ready'
  AND c.numTrials = 1
  AND c.labname IN (
    'laba04','laba04s','laba05','laba06','laba06s','project1'
  )
  AND c.uid LIKE 'user_%'
;
"""
conn.execute(create_datamart_sql)
conn.commit()

# ## Шаг 3: Загрузка datamart в Pandas и приведение типов
datamart = pd.read_sql(
    "SELECT * FROM datamart;",
    conn,
    parse_dates=["first_commit_ts", "first_view_ts"]
)
datamart.dtypes


uid                        object
labname                    object
first_commit_ts    datetime64[ns]
first_view_ts      datetime64[ns]
dtype: object

## Using Pandas methods, create two dataframes: test and control


In [13]:
# ## Шаг 4: Разделение на test и control группы
test    = datamart[datamart["first_view_ts"].notna()].copy()
control = datamart[datamart["first_view_ts"].isna()].copy()
# ## Шаг 5: Замена пропусков в control на среднее first_view_ts из test
mean_view_ts = test["first_view_ts"].mean()
control["first_view_ts"].fillna(mean_view_ts, inplace=True)
# ## Шаг 6: Сохранение test и control обратно в базу
test.to_sql     ("test",    conn, if_exists="replace", index=False)
control.to_sql  ("control", conn, if_exists="replace", index=False)

81

## Close the connection


In [14]:
# ## Шаг 7: Закрытие соединения
conn.close()